# حیوانات کے ماہر نظام کا نفاذ

[AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) سے ایک مثال۔

اس نمونہ میں، ہم ایک سادہ علم پر مبنی نظام نافذ کریں گے تاکہ کچھ جسمانی خصوصیات کی بنیاد پر ایک جانور کا تعین کیا جا سکے۔ نظام کو مندرجہ ذیل AND-OR درخت کے ذریعے ظاہر کیا جا سکتا ہے (یہ پورے درخت کا ایک حصہ ہے، ہم آسانی سے مزید قواعد شامل کر سکتے ہیں):

![](../../../../../../translated_images/ur/AND-OR-Tree.5592d2c70187f283.webp)


## ہمارا اپنا ماہر نظام شیل مع الٹی استدلال کے ساتھ

آئیے ایک آسان زبان کی تعریف کرنے کی کوشش کرتے ہیں جو پیداواری قواعد کی بنیاد پر علم کی نمائندگی کے لیے ہو۔ ہم قواعد کی تعریف کے لیے Python کلاسز کو کلیدی الفاظ کے طور پر استعمال کریں گے۔ بنیادی طور پر تین اقسام کی کلاسز ہوں گی:
* `Ask` ایک سوال کی نمائندگی کرتا ہے جو صارف سے پوچھنا ضروری ہوتا ہے۔ اس میں ممکنہ جوابات کا مجموعہ ہوتا ہے۔
* `If` ایک قاعدے کی نمائندگی کرتا ہے، اور یہ قاعدے کے مواد کو ذخیرہ کرنے کے لیے صرف ایک نحوی سہولت ہے۔
* `AND`/`OR` کلاسز درخت کی AND/OR شاخوں کی نمائندگی کے لیے ہیں۔ وہ بس اندر دلائل کی فہرست کو ذخیرہ کرتے ہیں۔ کوڈ کو آسان بنانے کے لیے، تمام فعالیت والد کلاس `Content` میں تعریف کی گئی ہے۔


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

ہمارے نظام میں، ورکنگ میموری میں **حقائق** کی فہرست **خصوصیت-قدر جوڑے** کے طور پر موجود ہوگی۔ علم کا ذخیرہ ایک بڑی لغت کے طور پر تعریف کیا جا سکتا ہے جو اعمال (نئے حقائق جو ورکنگ میموری میں شامل کیے جانے چاہئیں) کو شرائط کے ساتھ جوڑتا ہے، جو AND-OR اظہار کے طور پر ظاہر کی جاتی ہیں۔ نیز، کچھ حقائق کو `Ask` کیا جا سکتا ہے۔


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

پیچھے کی طرف استدلال کرنے کے لیے، ہم `Knowledgebase` کلاس کی تعریف کریں گے۔ اس میں شامل ہوں گے:  
* کام کرنے کی `memory` - ایک لغت جو خصوصیات کو اقدار سے میپ کرتی ہے  
* Knowledgebase کے `rules` اوپر دیے گئے فارمیٹ میں  

دو اہم طریقے ہیں:  
* `get` خاصیت کی قدر حاصل کرنے کے لیے، ضرورت پڑنے پر استدلال کرتے ہوئے۔ مثال کے طور پر، `get('color')` رنگ کے سلاٹ کی قدر حاصل کرے گا (اگر ضروری ہوا تو پوچھے گا، اور بعد میں استعمال کے لیے کام کرنے والی میموری میں قدر محفوظ کرے گا)۔ اگر ہم پوچھیں `get('color:blue')`، تو یہ رنگ کے بارے میں پوچھے گا، اور پھر رنگ کی بنیاد پر `y`/`n` قدر واپس کرے گا۔  
* `eval` اصل استدلال انجام دیتا ہے، یعنی AND/OR درخت کو عبور کرتا ہے، ذیلی اہداف کا اندازہ لگاتا ہے، وغیرہ۔


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

اب آئیے اپنے حیوانات کے علم کے ذخیرے کی تعریف کریں اور مشاورت انجام دیں۔ نوٹ کریں کہ یہ کال آپ سے سوالات کرے گی۔ آپ ہاں-نہ کے سوالات کے لیے `y`/`n` ٹائپ کرکے جواب دے سکتے ہیں، یا طویل متعدد انتخابی جوابات والے سوالات کے لیے نمبر (0..N) دے کر جواب دے سکتے ہیں۔


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## فارورڈ انفرنس کے لیے Experta کا استعمال

اگلے مثال میں، ہم علم کی نمائندگی کی لائبریریز میں سے ایک، [Experta](https://github.com/nilp0inter/experta) کے ذریعے فارورڈ انفرنس کو نافذ کرنے کی کوشش کریں گے۔ **Experta** ایک لائبریری ہے جو Python میں فارورڈ انفرنس سسٹمز بنانے کے لیے استعمال ہوتی ہے، جو کلاسیکی پرانے سسٹم [CLIPS](http://www.clipsrules.net/index.html) کے مماثل ڈیزائن کی گئی ہے۔

ہم اپنے طور پر بھی فارورڈ چیننگ کو بغیر کسی بڑی مسئلے کے نافذ کر سکتے تھے، لیکن سادہ نفاذ عموماً بہت زیادہ مؤثر نہیں ہوتے۔ زیادہ مؤثر اصول میچنگ کے لیے ایک خاص الگورتھم [Rete](https://en.wikipedia.org/wiki/Rete_algorithm) استعمال کیا جاتا ہے۔


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

ہم اپنے نظام کو ایک کلاس کے طور پر تعریف کریں گے جو `KnowledgeEngine` کی ذیلی کلاس ہو۔ ہر قاعدہ ایک الگ فنکشن کے ذریعے تعریف کیا جاتا ہے جس پر `@Rule` تشریح ہوتی ہے، جو بتاتی ہے کہ یہ قاعدہ کب فعال ہونا چاہیے۔ قاعدے کے اندر، ہم `declare` فنکشن کا استعمال کرتے ہوئے نئے حقائق شامل کر سکتے ہیں، اور ان حقائق کو شامل کرنے سے آگے کے استدلال کے انجن کے ذریعے مزید قواعد کو بلایا جائے گا۔


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

جب ہم نے ایک علم کا ذخیرہ متعین کر لیا، ہم اپنی ورکنگ میموری کو کچھ ابتدائی حقائق سے بھر دیتے ہیں، اور پھر استدلال انجام دینے کے لیے `run()` طریقہ کو کال کرتے ہیں۔ آپ دیکھ سکتے ہیں کہ نتیجے کے طور پر نئی استنباط شدہ حقائق ورکنگ میموری میں شامل ہو جاتی ہیں، جن میں جانور کے بارے میں آخری حقیقت بھی شامل ہے (اگر ہم تمام ابتدائی حقائق کو درست طریقے سے سیٹ کریں)۔


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ڈسکلیمر**:
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کے ذریعے ترجمہ کی گئی ہے۔ اگرچہ ہم درستگی کے لئے کوشاں ہیں، براہ کرم اس بات سے آگاہ رہیں کہ خودکار تراجم میں غلطیاں یا نقصانات ہو سکتے ہیں۔ اصل دستاویز اپنی مادری زبان میں ہی قابل اعتبار ذریعہ سمجھی جانی چاہیے۔ اہم معلومات کے لیے پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والے کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
